# Reappraisal Training For Linguistic Distancing and Emotion Regulation




## Mount Google Drive and Install Dependencies


### Setup
```bash
> pipenv shell  #Generates a new virtual environment based on Pipfile
> pipenv install # Installs the packages in Pipfile.lock (Use --dev) to also install dev packages
```

**Sources**
-  [Sentiment Analysis Text Classification Tutorial](https://www.youtube.com/watch?v=8N-nM3QW7O0)
- [Using Catalyst for Training Organization](https://github.com/catalyst-team/catalyst)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#! pip install transformers datasets wandb 
# NLTK only needed when trying to re-encode data

### Select GPU usage if one exists on the machine.

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import wandb
from datasets import ReadInstruction

os.chdir( '/content/drive/MyDrive/ldh')
# wandb.login()

# Enable GPU usage, if we can.
if torch.cuda.is_available():
    print("Enabling GPU usage")
    device = torch.device("cuda:0")
    print(device)
    IS_GPU = True
else:
    print("No GPU available, running on CPU")
    device = torch.device("cpu") # Note: macOS incompatible with NVIDIA GPUs
    IS_GPU = False
    


Enabling GPU usage
cuda:0


## Import and Encode LDH Data

In [ ]:
from src.LDHData import LDHData

data = LDHData()
data.load_training_data()
data.load_eval_data()

Training data loaded from disk.
Evaluation data loaded from disk.


In [ ]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')  
# Wrap tokenizer.
def tokenize(x):
    tokenized = tokenizer(x, add_special_tokens=True, padding="max_length", max_length=150)
    return tokenized
    
encoded_train = data.train_dataset['obj'].map(
    lambda ds: tokenize(ds['response']), batched=True, batch_size=16
)
encoded_train.set_format(type='torch', output_all_columns=True)

Loading cached processed dataset at /content/drive/MyDrive/ldh/src/training/obj/cache-9273660f280cce50.arrow


## Set Training Arguments and Train Model



In [ ]:
from transformers import TrainingArguments, Trainer
from src.ReappModel import ReappModel
# Define the parameers under which the model will be trained.
# By default, uses an AdamW optimizer w/ linear warmup.
model = ReappModel()

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,
    evaluation_strategy="steps",
    logging_steps=100,
    save_steps=100,
    # label_names=['score'],
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    run_name="wandb-test-run-obj"
  )

##optimizers (Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR, optional) – 

In [ ]:
encoded_train_split, encoded_val_split = encoded_train.shuffle().select(range(10)).train_test_split(test_size=0.15).values()

trainer = Trainer(
  model=model.to(device),
  args=training_args,                  
  train_dataset=encoded_train_split,
  eval_dataset=encoded_val_split        
)    

trainer.train()

Loading cached shuffled indices for dataset at /content/drive/MyDrive/ldh/src/training/obj/cache-23250cc08b164e0d.arrow
W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.
/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


TypeError: ignored

# Scratch

In [ ]:
from src.ReappModel import ReappModel
model = ReappModel()


TypeError: ignored

In [ ]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [41]:
from torch import nn
classifier_head = nn.Sequential(*list(model.children())[-3:])

In [43]:
classifier_head

Sequential(
  (0): Linear(in_features=768, out_features=768, bias=True)
  (1): Linear(in_features=768, out_features=2, bias=True)
  (2): Dropout(p=0.2, inplace=False)
)

In [44]:
new_model = nn.Sequential(*list(model.children())[:-3])

In [45]:
new_model

Sequential(
  (0): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_features=76